In [5]:
!pip install description

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement description (from versions: none)
ERROR: No matching distribution found for description


In [7]:
!pip install featuretools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.0 MB/s 
     |████████████████████████████████| 207 kB 38.7 MB/s 
     |████████████████████████████████| 280 kB 7.7 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [5]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 4.7 MB/s 


In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(font_scale=2.2)
plt.style.use('seaborn')

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import f1_score
import lightgbm as lgb
import itertools
import xgboost as xgb
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

import shap
from tqdm import tqdm
import featuretools as ft
import warnings
warnings.filterwarnings('ignore')

import time


Woodwork may not support Python 3.7 in next non-bugfix release.
Featuretools may not support Python 3.7 in next non-bugfix release.


Check datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

direc='/content/drive/MyDrive/주피터_대피소/kaggle&github/Kaggle/4.Costa_Rican_Household_Poverty/'

df_train=pd.read_csv(direc+'train.csv')
df_test=pd.read_csv(direc+'test.csv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# df_train = pd.read_csv('../input/train.csv')
# df_test = pd.read_csv('../input/test.csv')

In [3]:
print('df_train shape : ', df_train.shape, '  ', 'df_test_shape : ',df_test.shape)

df_train shape :  (9557, 143)    df_test_shape :  (23856, 142)


Make description df

NameError: ignored

Check null data

In [6]:
total = df_train.isnull().sum().sort_values(ascending=False)

percent = 100 * (df_train.isnull().sum() / df_train.isnull().count().sort_values(ascending=False))

missing_df = pd.concat([total, percent], axis=1, keys = ['Total', 'Percent'])

missing_df.head()

,Total,Percent
rez_esc,7928,82.954902
v18q1,7342,76.823271
v2a1,6860,71.779847
SQBmeaned,5,0.052318
meaneduc,5,0.052318


Fill Missin gvalues

In [ ]:
#if education is yes and person is head of households, fill with escolari

df_train.loc[(df_train['edjefa'] == 'yes') & (df_train['parentesco1'] ==1), 'edjefa'] = df_train.loc[(df_train['edjefa'] == 'yes') & (df_train['parentesco1'] ==1), 'escolari']

df_train.loc[(df_train['edfeje'] =='yesa') & (df_train['parentesco1'] ==1), 'edjefe'] = df_train.loc[(df_train['edjefe'] == 'yes') & (df_train['parentesco1'] ==1), 'escolari']

df_test.loc[(df_test['edjefa'] == "yes") & (df_test['parentesco1'] == 1), "edjefa"] = df_test.loc[(df_test['edjefa'] == "yes") & (df_test['parentesco1'] == 1), "escolari"]
df_test.loc[(df_test['edjefe'] == "yes") & (df_test['parentesco1'] == 1), "edjefe"] = df_test.loc[(df_test['edjefe'] == "yes") & (df_test['parentesco1'] == 1), "escolari"]

#this filed is supposed to be interaction btw gender and escolari, but it isn;t clear what yes means, let's fill in with 4

df_train.loc[df_train['edjefa'] =='yes', 'edjefa'] =4
df_train.loc[df_train['edjefe'] == 'yes', 'edfeje'] = 4

df_test.loc[df_test['edjefa'] =='yes', 'edjefa'] =4
df_test.loc[df_test['edjefe'] =='yes', 'edjefe'] =4

#create featur with max education of either head of household

df_train['dejef'] = np.max(df_train[['edjefa','edjefe']], axis=1)
df_test['dejef'] = np.max(df_test[['edjefa','edjefe']], axis=1)

#fix some inconsistencies in the data - some rows indicate both that the household does and does not have a toilet,
#if there is no water we'll assume they do not

df_train.loc[(df_train['v14a']==1) & (df_train.sanitario1 ==1) & (df_train['abastaguano'] == 0 ), 'v14a'] =0
df_train.loc[(df_train.v14a ==1) & (df_train.sanitario1 ==1) & (df_train.abastaguano == 0 ), 'sanitario1']=0

df_test.loc[(df_test.v14a ==  1) & (df_test.sanitario1 ==  1) & (df_test.abastaguano == 0), "v14a"] = 0
df_test.loc[(df_test.v14a ==  1) & (df_test.sanitario1 ==  1) & (df_test.abastaguano == 0), "sanitario1"] = 0

rez_esz, SQBmeaned

In [ ]:
df_train.rez_esc.fillna(0, inpalce=True)
df_test = df_test.rez_esc.fillna(0)

In [ ]:
df_train['SQBmeaned'].fillna(0, inplace=True)
df_test['SQBmeaned'].fillna(0, inplace=True)

meaneduc

In [ ]:
df_train.meaneduc.fillna(0, inplace=True)
df_test.meaneduc.fillna(0, inplace=True)

v18q1

In [ ]:
df_train['v18q'].value_counts()

In [ ]:
df_train.loc[df_train['v18q'] ==1].value_counts()

In [ ]:
df_train.loc[df_train['v18q'] ==0, 'v18q'].value_counts()

In [ ]:
df_train['v18q'].fillna(0, inplace=True)
df_test['v18q'].fillna(0, inplace=True)

In [ ]:
df_train['topivivi3'].value_counts()

In [ ]:
sns.kdeplot(df_train.loc[df_train['tipovivi3'] ==1, 'v2a1'], label='Monthly rent payment of household(rented=1)' )

sns.kdeplot(df_train.loc[df_train['tipovivi3'] ==0, 'v2a1'], label='monthly rent payment of household(rented=0')

plt.xscale('log')
plt.show()

In [ ]:
df_train['v2a1'].fillna(0, inplace=True)
df_test['v2a1'].fillna(0, inplace=True)

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)

percent = 100 * (df_train.isnull().sum() / df_train.isnull().count()).sort_values(ascending=False)

#can also apply 100*(df_train.isnull().sum() / len(df_train.isnull()))

missing_df =  pd.concat([total,percent], axis=1, keys=['Total', 'Percent'])

missing_df.head()

In [ ]:
total = df_test.isnull().sum().sort_values(ascending=False)

percent = 100 * (df_test.isnull().sum() / df_test.isnull().count()).sort_values(ascending=False)

#can also apply 100*(df_train.isnull().sum() / len(df_train.isnull()))

missing_df =  pd.concat([total,percent], axis=1, keys=['Total', 'Percent'])

missing_df.head()

#Feature engineering

Object Features

In [ ]:
features_object = [col for col in df_train.columns if df_train[col].dtype == 'object']

dependecncy

In [ ]:
df_train['dependency'] = np.sqrt(df_train['SQBdependency'])

df_test['dependency'] = np.sqrt(df_test['SQBdependency'])

edjefe

In [ ]:
def replace_binary(x) :
  if x == 'yes' :
    return 1
  elif x == 'no' :
    return 0
  else :
    return np.nan

df_train['edjefe'] = df_train['edjefe'].apply(replace_binary).astype(float)
df_test['edjefe'] = df_test['edjefe'].apply(replace_binary).astype(float)

df_train['edjefa'] = df_test['edjefa'].apply(replace_binary).astype(float)
df_test['edjefa'] = df_test['edjefa'].apply(reaplce_binary).astype(float)

In [ ]:
#create feature with max education of either head of household
df_train['edjef'] = np.max(df_train[['edjefe','edjefa']],axis=1)
df_test['edjef'] = np.max(df_test[['edjefe','edjefa']],axis=1)

roof and electricity

In [ ]:
df_train['roof_waste_material'] =np.nan
df_test['roof_waste_material'] =np.nan
df_train['electricity_other'] =np.nan
df_test['electricity_other'] =np.nan

def fill_roof_exception(x) :
  if (x['techozinc'==0]) & (x['techoentrepiso'] ==0) & (x['techocane']==0) & (x['techootro']==0) :
    return 1
  else :
    return 0

def fill_no_electricity(x) :
  if (x['public'] ==0) & (x['planpri']==0) & (x['noelec'] ==0) & (x['coopele']==0) :
    return 1
  else :
    return 0


df_train['roof_waste_material'] = df_train.apply(lambda x : fill_roof_exception(x), axis=1)
df_test['roof_waste_material'] = df_test.apply(lambda x : fill_roof_exception(x), axis=1)
df_train['electiricity_other'] = df_train.apply(lambda x : fill_no_electricity(x), axis=1)
df_test['electiricity_other'] = df_test.apply(lambda x : fill_no_electricity(x), axis=1)

In [ ]:
bin_cat_feature = [col for col in df_train.columns if df_train[col].value_counts().shape[0] ==2]

2-3. Make new FEatures using continuous feature

In [ ]:
continuous_features = [col for col in df_train.columns if col not in bin_cat_feature]

continuous_features = [col for col in continuous_features if col not in features_object]

continuous_features = [col for col incontinuous_features if col not in ['Id','Target','idhogar']]

In [ ]:
print('There are {} continuous features'.format(len(continuous_features)))

for col in continuous_features :
  print('{} {}'.foramt(col, description.loc[description['varname']==col, 'description'].values))

In [ ]:
df_train['edjef'].value_counts()

In [ ]:
df_train.drop('tamhog',axis=1, inplace=True)

df_test.drop('tamhog', axis=1, inplace=True)

Squared features

- There are many squared features. Actually, tree models like lightgbm dont need them. But at this kernel, I want to use lightgbm as feature filter model and set entity - embedding as classifier. So let's keep them.

Family features
- hogar_nin, hogar_adul, hogar_mayor, hogar_total, r4h1, r4h2, r4h3, r4m1, r4m2, r4m3, r4t1, r4t2, r4t3, tmbhog, tamvid, rez_esc, escolari
- Family size features (substract, ratio)

In [ ]:
df_train['adult'] = df_train['hogar_adul'] - df_train['hogar_mayor']
df_train['dependency_count'] = df_train['hogar_nin'] + df_train['hogar_mayor']
df_train['dependency'] = df_train['dependency_count'] / df_train['adult']
df_train['child_percent'] = df_train['hogar_nin'] / df_train['hogar_total']
df_train['elder_percent'] = df_train['hogar_mayor'] / df_train['hogar_total']
df_train['adult_percent'] = df_train['hogar_adul'] / df_train['hogar_total']
df_train['males_younger_12_years_percent'] = df_train['r4h1'] / df_train['hogar_total']
df_train['males_older_12_years_percent'] = df_train['r4h2'] / df_train['hogar_total']
df_train['males_percent'] = df_train['r4h3'] / df_train['hogar_total']
df_train['females_younger_12_years_percent'] = df_train['r4m1'] / df_train['hogar_total']
df_train['females_older_12_years_percent'] = df_train['r4m2'] / df_train['hogar_total']
df_train['females_percent'] = df_train['r4m3'] / df_train['hogar_total']
df_train['persons_younger_12_years_percent'] = df_train['r4t1'] / df_train['hogar_total']
df_train['persons_older_12_years_percent'] = df_train['r4t2'] / df_train['hogar_total']
df_train['persons_percent'] = df_train['r4t3'] / df_train['hogar_total']

In [ ]:
df_test['adult'] = df_test['hogar_adul'] - df_test['hogar_mayor']
df_test['dependency_count'] = df_test['hogar_nin'] + df_test['hogar_mayor']
df_test['dependency'] = df_test['dependency_count'] / df_test['adult']
df_test['child_percent'] = df_test['hogar_nin'] / df_test['hogar_total']
df_test['elder_percent'] = df_test['hogar_mayor'] / df_test['hogar_total']
df_test['adult_percent'] = df_test['hogar_adul'] / df_test['hogar_total']
df_test['males_younger_12_years_percent'] = df_test['r4h1'] / df_test['hogar_total']
df_test['males_older_12_years_percent'] = df_test['r4h2'] / df_test['hogar_total']
df_test['males_percent'] = df_test['r4h3'] / df_test['hogar_total']
df_test['females_younger_12_years_percent'] = df_test['r4m1'] / df_test['hogar_total']
df_test['females_older_12_years_percent'] = df_test['r4m2'] / df_test['hogar_total']
df_test['females_percent'] = df_test['r4m3'] / df_test['hogar_total']
df_test['persons_younger_12_years_percent'] = df_test['r4t1'] / df_test['hogar_total']
df_test['persons_older_12_years_percent'] = df_test['r4t2'] / df_test['hogar_total']
df_test['persons_percent'] = df_test['r4t3'] / df_test['hogar_total']

In [ ]:
df_train['males_younger_12_years_in_household_size'] = df_train['r4h1'] / df_train['hhsize']
df_train['males_older_12_years_in_household_size'] = df_train['r4h2'] / df_train['hhsize']
df_train['males_in_household_size'] = df_train['r4h3'] / df_train['hhsize']
df_train['females_younger_12_years_in_household_size'] = df_train['r4m1'] / df_train['hhsize']
df_train['females_older_12_years_in_household_size'] = df_train['r4m2'] / df_train['hhsize']
df_train['females_in_household_size'] = df_train['r4m3'] / df_train['hogar_total']
df_train['persons_younger_12_years_in_household_size'] = df_train['r4t1'] / df_train['hhsize']
df_train['persons_older_12_years_in_household_size'] = df_train['r4t2'] / df_train['hhsize']
df_train['persons_in_household_size'] = df_train['r4t3'] / df_train['hhsize']

In [ ]:
df_test['males_younger_12_years_in_household_size'] = df_test['r4h1'] / df_test['hhsize']
df_test['males_older_12_years_in_household_size'] = df_test['r4h2'] / df_test['hhsize']
df_test['males_in_household_size'] = df_test['r4h3'] / df_test['hhsize']
df_test['females_younger_12_years_in_household_size'] = df_test['r4m1'] / df_test['hhsize']
df_test['females_older_12_years_in_household_size'] = df_test['r4m2'] / df_test['hhsize']
df_test['females_in_household_size'] = df_test['r4m3'] / df_test['hogar_total']
df_test['persons_younger_12_years_in_household_size'] = df_test['r4t1'] / df_test['hhsize']
df_test['persons_older_12_years_in_household_size'] = df_test['r4t2'] / df_test['hhsize']
df_test['persons_in_household_size'] = df_test['r4t3'] / df_test['hhsize']

In [ ]:
df_train['overcrowding_room_and_bedroom'] = (df_train['hacdor'] + df_train['hacapo'])/2
df_test['overcrowding_room_and_bedroom'] = (df_test['hacdor'] + df_test['hacapo'])/2

In [ ]:
df_train['escolari_age'] = df_train['escolari']/df_train['age']
df_test['escolari_age'] = df_test['escolari']/df_test['age']

df_train['age_12_19'] = df_train['hogar_nin'] - df_train['r4t1']
df_test['age_12_19'] = df_test['hogar_nin'] - df_test['r4t1']

In [ ]:
df_train['phones-per-capita'] = df_train['qmobilephone'] / df_train['tamviv']
df_train['tablets-per-capita'] = df_train['v18q1'] / df_train['tamviv']
df_train['rooms-per-capita'] = df_train['rooms'] / df_train['tamviv']
df_train['rent-per-capita'] = df_train['v2a1'] / df_train['tamviv']

In [ ]:
df_test['phones-per-capita'] = df_test['qmobilephone'] / df_test['tamviv']
df_test['tablets-per-capita'] = df_test['v18q1'] / df_test['tamviv']
df_test['rooms-per-capita'] = df_test['rooms'] / df_test['tamviv']
df_test['rent-per-capita'] = df_test['v2a1'] / df_test['tamviv']

- You can see that ' total person in the household' != '# fo total individuals in the household'.

- somewhat weired. but for now i will keep it.

In [ ]:
(df_train['hogar_total'] == df_train['r4t3']).sum()

Rent per family features

- i will reduce the # features using shap, so let's generate many features.

In [ ]:
family_size_features = ['adult', 'hogar_adul', 'hogar_mayor', 'hogar_nin', 'hogar_total', 'r4h1', 
                        'r4h2', 'r4h3', 'r4m1', 'r4m2', 'r4m3', 'r4t1', 'r4t2', 'r4t3', 'hhsize']

new_feats = []

for col in family_size_features :
  new_col_name = 'new_{}_per_{}'.format('v2a1',col)
  new_feats.append(new_col_name)
  df_train[new_col_name] = df_train['v2a1'] / df_train[col]
  df_test[new_col_name] = df_test['v2a1'] / df_test[col]

In [ ]:
for col in new_feats :
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace([np.inf], np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

Room per family features

In [ ]:
new_feats = []

for col in family_size_features :
  new_col_name = 'new_{}_per_{}'.format('rooms',col)
  new_feats.append(new_col_name)

  df_train[new_col_name] = df_train['rooms'] / df_train[col]
  df_test[new_col_name] = df_test['rooms'] / df_test[col]


for col in new_feats :
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace=([np.inf], np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

BedRoom per faily features

In [ ]:
new_feats = []
for col in family_size_features:
    new_col_name = 'new_{}_per_{}'.format('bedrooms', col)
    new_feats.append(new_col_name)
    df_train[new_col_name] = df_train['bedrooms'] / df_train[col]
    df_test[new_col_name] = df_test['bedrooms'] / df_test[col]

for col in new_feats:
    df_train[col].replace([np.inf], np.nan, inplace=True)
    df_train[col].fillna(0, inplace=True)
    
    df_test[col].replace([np.inf], np.nan, inplace=True)
    df_test[col].fillna(0, inplace=True)

In [ ]:
print(df_train.shape, df_test.shape) 
# To check the same number of features between train and test (target is there in train)

Tabulet per family features

In [ ]:
new_feats = []

for col in family_size_features :
  new_col_name  = 'new_{}_per_{}'.format('v18q1', col)
  new_feats.append(new_col_name)

  df_train[new_col_name] = df_train ['v18q1'] / df_train[col]
  df_test[new_col_name] = df_test['v18q1'] / df_test[col]

for col in new_feats :
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace=([np.inf],np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

phone per family features

In [ ]:
new_feats = []
for col in family_size_features:
    new_col_name = 'new_{}_per_{}'.format('qmobilephone', col)
    new_feats.append(new_col_name)
    df_train[new_col_name] = df_train['qmobilephone'] / df_train[col]
    df_test[new_col_name] = df_test['qmobilephone'] / df_test[col]

for col in new_feats:
    df_train[col].replace([np.inf], np.nan, inplace=True)
    df_train[col].fillna(0, inplace=True)
    
    df_test[col].replace([np.inf], np.nan, inplace=True)
    df_test[col].fillna(0, inplace=True)

rez_esc(years behond in school) per family features

In [ ]:
new_feats = []
for col in family_size_features:
    new_col_name = 'new_{}_per_{}'.format('rez_esc', col)
    new_feats.append(new_col_name)
    df_train[new_col_name] = df_train['rez_esc'] / df_train[col]
    df_test[new_col_name] = df_test['rez_esc'] / df_test[col]

for col in new_feats:
    df_train[col].replace([np.inf], np.nan, inplace=True)
    df_train[col].fillna(0, inplace=True)
    
    df_test[col].replace([np.inf], np.nan, inplace=True)
    df_test[col].fillna(0, inplace=True)

In [ ]:
df_train['rez_esc_age'] = df_train['rez_esc'] / df_train['age']
df_train['rez_esc_escolari'] = df_train['rez_esc'] / df_train['escolari']

df_test['rez_esc_age'] = df_test['rez_esc'] / df_test['age']
df_test['rez_esc_escolari'] = df_test['rez_esc'] / df_test['escolari']

Rich features
- i think the more richer, the larger number of phones and tablet

In [ ]:
df_train['tabulet_*_qmobilephone'] = df_train['v18q1']*df_train['qmobilephone']

df_test['tabulet_*_qmobilephone'] = df_test['v18q1']*df_test['qmobilephone']

- wall, roof, floor may be key factor.
- let's multiply each of them. Because they are binary cat features, so multification of each features generates new categorical features

In [ ]:
# wall and roof

for col1 in ['epared1', 'epared2', 'epared3'] :
  for col2 in ['etecho1','etecho2','etecho3'] :
    new_col_name = 'new_{}x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col] * df_train[col2]
    df_test[new_col_name] = df_test[col] * df_test[col2]


#wall and floor
for col1 in ['epared1', 'epared2', 'epared3']:
    for col2 in ['eviv1', 'eviv2', 'eviv3']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]

# roof and floor
for col1 in ['etecho1', 'etecho2', 'etecho3']:
    for col2 in ['eviv1', 'eviv2', 'eviv3']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]

combinating using three features

In [ ]:
for col1 in ['epared1','epared2','epared3'] :
  for col2 in ['etecho1','etecho2','etecho3'] :
    for col3 in ['eviv1','eviv2','eviv3'] :
      new_col_name = 'new_{}_*_{}_*_{}'.format(col1, col2, col3)

      df_train[new_col_name] = df_train[col1] * df_train[col2] * df_train[col3]

      df_test[new_col_name] = df_test[col1] * df_test[col2] * df_test[col3]

      

In [ ]:
print(df_train.shape, df_test.shape)

- i want to mix electricity and energy features --> energy features

In [ ]:
for col1 in ['public','planpri','noelec','coopele'] :
  for col2 in ['energcocinar1','energcocinar2','energcocinar3','energcocinar4'] :
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1]  * df_train[col2]

i want to mix toilet and rubbish disposal features --> other_infra features

In [ ]:
for col1 in ['sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6']:
    for col2 in ['elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 'elimbasu5', 'elimbasu6']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]

mix water provision features --> water features

In [ ]:
for col1 in ['abastaguadentro', 'abastaguafuera', 'abastaguano']:
    for col2 in ['sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]

In [ ]:
print(df_train.shape, df_test.shape)

mix education and area features --> education_zone_features

In [ ]:
for col1 in ['area1', 'area2']:
    for col2 in ['instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]

In [ ]:
for col1 in ['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6']:
    for col2 in ['instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]

multiply television / mobilephone / computer / tabulet / regrigerator --> electronic features

In [ ]:
df_train['electronics'] = df_train['computer'] * df_train['mobilephone'] * df_train['television'] * df_train['v18q'] * df_train['refrig']
df_test['electronics'] = df_test['computer'] * df_test['mobilephone'] * df_test['television'] * df_test['v18q'] * df_test['refrig']

df_train['no_appliances'] = df_train['refrig'] + df_train['computer'] + df_train['television'] + df_train['mobilephone']
df_test['no_appliances'] = df_test['refrig'] + df_test['computer'] + df_test['television'] + df_test['mobilephone']

mix all material of roof, floor, wall

In [ ]:
for col1 in ['paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother']:
    for col2 in ['pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera']:
      new_col_name = 'new_{}_*_{}'.format(col1,col2)
      df_train[new_col_name] = df_train[col1] * df_train[col2]
      df_test[new_col_name] = df_test[col1] * df_test[col2]


for col1 in ['pisomoscer', 'pisocemento','pisoother','pisonatur','pisonotiene','pisomadera'] :
  for col2 in ['techozinc','techoentrepiso','techocane','techootro'] :
    new_col_name = 'new_{}_*{}'.format(col1,col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]


for col1 in ['paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother']:
    for col2 in ['techozinc', 'techoentrepiso', 'techocane', 'techootro']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]        
        
for col1 in ['paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother']:
    for col2 in ['pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera']:
        for col3 in ['techozinc', 'techoentrepiso', 'techocane', 'techootro']:
            new_col_name = 'new_{}_x_{}_x_{}'.format(col1, col2, col3)
            df_train[new_col_name] = df_train[col1] * df_train[col2] * df_train[col3]
            df_test[new_col_nam

In [ ]:
print(df_train.shape, df_test.shape)

Remove feature with only one value

In [ ]:
cols_with_only_one_value = []
for col in df_train.columns :
  if col == 'Target' :
    continue
  if df_train[col].value_counts().shape[0] == 1 or df_test[col].value_counts().shape[0] ==1 :
    print(col)

    cols_with_only_one_value.append(col)

In [ ]:
df_train.drop(cols_with_only_one_value, axis=1, inpalce=True)
df_test = df_test.drop(cols_with_only_one_value, axis=1)

Check whether both train and test have same features

In [ ]:
cols_train = np.array(sorted([col for col in df_train.columns if col != 'Targe']))

cols_test = np.array(sorted(df_test.columns))

In [ ]:
(cols_train == cols.test).sum() == len(cols_train)

aggregation features
- in this competition, each samples are member of specific household. So let's aggregate based on ***'idhogar'*** values

agg for family features

In [ ]:
def max_min(x) :
  return x.max()-x.min()

In [ ]:
agg_train = pd.DataFrame()
agg_test = pd.DataFrame()

for item in tqdm(family_size_features) :
  for i, function in enumerate(['mean','std','min','max','sum','count', max_min]) :
    group_train = df_train[item].groupby(df_train['idhogar']).agg(funcion)

    group_test = df_test[item].groupby(df_train['idhogar']).agg(function)

    if i==6 :
      new_col = item + '_new_' + 'max_min'
    else :
      new_col = item + '_new_' + function
    
    agg_train[new_col] = group_train
    agg_test[new_col] = group_test

print('new agg train set has {} rows, and {} features'.format(agg_train.shape[0], agg_train.shape[1]))

print('new agg test set has {} rows, and {} features'.format(agg_test.shape[0], agg_test.shape[1]))

In [ ]:
aggr_list = ['rez_esc', 'dis', 'male', 'female', 
                  'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
                  'parentesco2', 'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 
                  'parentesco11', 'parentesco12',
                  'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',
                 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 'refrig', 'television', 'mobilephone',
            'area1', 'area2', 'v18q', 'edjef']

In [ ]:
for item in tqdm(aggr_list) :
  for function in ['count','sum'] :
    group_train = df_train[item].groupby(df_train['idhogar']).agg(function)
    group_test = df_test[item].groupby(df_test['idhogar']).agg(function)

    new_col = item + '_new1_' + function
    agg_train[new_col] = group_train
    agg_test[new_col] = group_test

print('neww aggregate train set ha s{} rows, and {} features.'.format(agg_train.shape[0], agg_train.shape[1]))
print('new aggregate test set has {} rows and {} features'.format(agg_test.shape[0], agg_test.shape[1]))

In [ ]:
aggr_list = ['escolari', 'age', 'escolari_age', 'dependency', 'bedrooms', 'overcrowding', 'rooms', 'qmobilephone', 'v18q1']

for item in tqdm(aggr_list):
    for function in ['mean','std','min','max','sum', 'count', max_min]:
        group_train = df_train[item].groupby(df_train['idhogar']).agg(function)
        group_test = df_test[item].groupby(df_test['idhogar']).agg(function)
        if i == 6:
            new_col = item + '_new2_' + 'max_min'
        else:
            new_col = item + '_new2_' + function
        agg_train[new_col] = group_train
        agg_test[new_col] = group_test

print('new aggregate train set has {} rows, and {} features'.format(agg_train.shape[0], agg_train.shape[1]))
print('new aggregate test set has {} rows, and {} features'.format(agg_test.shape[0], agg_test.shape[1]))

In [ ]:
agg_test = agg_test.reset_index()
agg_train = agg_train.reset_index()

train_agg = pd.merge(df_train, agg_train, on='idhogar')
test = pd.merge(df_test, agg_test, on='idhogar')

#fill all na as 0
train_agg.fillna(0, inplace=True)
test.fillna(0, inplace=True)

print('train shape : ', train_agg.shape, 'test_shape', test.shape)

In [ ]:
aggr_list = ['rez_esc', 'dis', 'male', 'female', 
                  'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
                  'parentesco2', 'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 
                  'parentesco11', 'parentesco12',
                  'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',
                 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 'refrig', 'television', 'mobilephone',
            'area1', 'area2', 'v18q', 'edjef']
  

for lugar in ['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6']:
    group_train = df_train[[lugar, 'idhogar'] + aggr_list].groupby([lugar, 'idhogar']).sum().reset_index()
    group_train.columns = [lugar, 'idhogar'] + ['new3_{}_idhogar_{}'.format(lugar, col) for col in group_train][2:]

    group_test = df_test[[lugar, 'idhogar'] + aggr_list].groupby([lugar, 'idhogar']).sum().reset_index()
    group_test.columns = [lugar, 'idhogar'] + ['new3_{}_idhogar_{}'.format(lugar, col) for col in group_test][2:]

    train_agg = pd.merge(train_agg, group_train, on=[lugar, 'idhogar'])
    test = pd.merge(test, group_test, on=[lugar, 'idhogar'])
    
print('train shape:', train_agg.shape, 'test shape:', test.shape)

In [ ]:
cols_nums = ['age', 'meaneduc', 'dependency', 
             'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total',
             'bedrooms', 'overcrowding']

for function in tqdm(['mean','std','min','max','sum','count', max_min] ):
  for lugar in ['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6']:
    group_train = df_train[[lugar, 'idhogar'] + aggr_list].groupby([lugar, 'idghoar']).agg(function).reset_index()
    group_test = df_test[[lugar, 'idhogar'] + aggr_list].groupby([lugar,'idhogar']).agg(function).reset_index()

    group_train.columns = [lugar,'idhogar'] + ['new5_{}_idhogar_{]_{}'.foramt(lugar,col,function) for col in group_train] [2:]
    group_test.columns = [lugar,'idhogar'] + ['new5_{}_idhogar_{]_{}'.foramt(lugar,col,function) for col in group_test] [2:]

    train_agg = pd.merge(train_agg, group_train, on=[lugar,'idhogar'])
    test = pd.merge(test, group_test, on=[lugar, 'idhogar'])

ptinrt('train shape : ', train_agg.shape, 'test shape : ', test_agg.shape)


according to data descriptions, only the heads of household are used in scroing
- all household members are included in test + the sample subsmiions, but onlt heads of households are scores.

In [ ]:
train = train_agg.query('parentesco ==1')

In [ ]:
train['dependency'].replace(np.inf, 0, inplace=True)
test['dependency'].repalce(np.inf, 0, inplace=True)

In [ ]:
submission = test[['Id']]

train.drop(['idhogar','Id', 'agesq', 'hogar_adul', 'SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned'],axis=1, inplace=True)

test.drop(['idhogar','Id',  'agesq', 'hogar_adul', 'SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned'], aixs=1, inplace=True)

correaltions = train.corr()
correaltions = correlations['Target'].sort_values(ascending=False)

In [ ]:
print('final_data_size', train.shape, test.shape)

In [ ]:
print(f'the most 20 posirive feature : \n {corrlations.head(20)}')


In [ ]:
print(f'the most 20 negative feature : \n {corrlations.tail(20)}')


#4. feature selection using shap

In [ ]:
bin_cat_features = [col for col intrain.column if train[col].value_counts().shape[0] ==2]

object_features = ['edjefe','edjefa']

categorical_feats = bin_cat_features + object_features

In [ ]:
def evaluate_marco_f1_lgb(truth, predictions) :
  pred_lables = predictions.reshape(len(np.unique(truth)), -1).argmax(axis=0)
  f1 = f1_score(truth, pred_labels, average='macro')
  return ('macrof1',f1,True)

In [ ]:
y=trian['Target']
trian.drop(['Target'],axis=1, inplace=True)

In [ ]:
def print_execution_time(start) :
  end=time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, secons = divmod(rem, 60)
  print('*'*20, 'Exucutino ended in {:0>2}, {:05.2f}s'.format(int(hours),int(minutes),int(seconds), '*'*20))

In [ ]:
def extract_good_features_unsing_shap_lgb(params, seed) :
  clf = lgb.LGBMClassifier(objective='multiclass',
                           random_state=42,
                           max_depth=params['max_depth'],
                           learning_rate=params['learning_rate'],
                           silent=True,
                           metric='multi_logloss',
                           n_jobs=-1,
                           n_estimators=10000,
                           class_weight='balanced',
                           colsamples_bytree=parmas['colsample_bytree'],
                           min_split_gain=params['min_split_gain'],
                           baggin_freq=params['bagging_freq'],
                           min_child_weight=params['min_child_weight'],
                           num_leaves=params['num_leaves'],
                           subsample=params['subsample'],
                           reg_alpha=params['reg_alpha'],
                           reg_labmda = parmas['reg_lambda'],
                           num_class = len(np.unique(y)),
                           bagging_seed=seed,
                           seed=seed)
  kfold=5
  kf=StratifiedKFold(n_splits=kfold, shuffle=True)
  feat_importance_df = pd.DataFrame()

  for i, (train_index, test_index) in enumerate(kf.split(train,y)) :
    print('='*30, '{} of {} folds'.format(i+1, kfold), '='*30)
    start=time.time()
    x_train,x_val = train.loc[train_index], train.loc[test_index]
    y_train, y_val = y.loc[train_index], y.loc[test_index]
    clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_val, y_val)],
            eval_metric=evaluate_macrof1_lgb, categorical_feature=categorical_feats, early_stopping_rounds=500, verbose=500)
    
    shap_values = shap.TreeExplainer(clf.booster_).shap_values(x_train)
    fold_importance_df = pd.DataFrame()
    fold_importance_df['feature'] = x.train.columns
    fold_importance_df['shap_values'] = abs(np.array(shap_values)[:,:].mean(1).mean(0))
    fold_importance_df['feat_imp'] = clf.feature_importances_
    feat_importance_df = pd.concat([feat_importance_df, fold_importance_df])
    print_execution_time(start)

  feat_importance_df_shap = feat_importance_df.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

  return feat_importance_df_shap


In [ ]:
total_shap_df = pd.DataFrame()

num_iterations = 50
for seed in range(num_iterations) :
  print('#'*40, '{} fo {} iterations'.format(seed+1, num_iterstions), '#' * 40)
  params = {'max_depth' : np.random.choice([5,6,7,8,10,12,-1]),
            'learning_rate' : np.random.rand() * 0.02,
            'colsample_bytree' : np.random.rand() * (-0.5) + 0.5,
            'subsample' : np.random.rand() * (1-0.5) + 0.5,
            'min_split_gain' : np.random.rand() * 0.2
            'num_leaves': np.random.choice([32, 48, 64]),
            'reg_alpha': np.random.rand() * 2,
            'reg_lambda': np.random.rand() *2,
            'bagging_freq': np.random.randint(4) +1,
            'min_child_weight': np.random.randint(100) + 20
            }
  temp_shap_df = extract_good_features_using_shap_LGB(params, SEED)
  total_shap_df = pd.concat([total_shap_df, temp_shap_df])}

In [ ]:
shap_sorted_df = total_shap_df.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

feat_imp_sorted_df = total_shap_df.groupby('feature').mean().sort_values('feat_imp', ascending=False).reset_index()

features_top_shap = shap_sorted_df['feature'][:500]
features_top_feat_imp = feat_imp_sorted_df['feature'][:500]
top_features = pd.Series(features_top_shap.tolist() + features_top_feat_imp.tolist())
top_features = top_features.unique()

#4. Model development

In [ ]:
new_train = train[top_features].copy()
new_test = test[top_features].copy()

In [ ]:
print('new_train_shape : ', new_train.shape, 'new_test_shape :', new_test.shape)

In [ ]:
new_categorical_feats = [col for col in top_features is col in categorical_feats]

out of folds of Light gradient boosting

In [ ]:
def LGB_OOF(params, categorical_feats, n_folds, seed=42) :
  clf=lgb.LGBMClassifier(objective='multiclass',
                         random_state=42,
                         max_depth=params['max_depth'],
                         learing_rate=params['learning_rate'],
                         silent=True,
                         metric='multi_logloss',
                         n_jobs=-1,
                         n_estimators=10000,
                         class_weight='balanced',
                         colsample_bytree = params['colsample_bytree'],
                         min_split_gain=params['min_split_gain'],
                         bagging_freq=params['bagging_freq'],
                         min_child_weight=params['min_child_weight'],
                         num_leaves=  params['num_leaves'],
                         sabsample = params['subsample'],
                         reg_alpha=params['reg_alpha'],
                         reg_lambda=params['reg_lambda'],
                         num_class = len(np.unique(y)),
                         bagging_seed= seed,
                         seed=seed)
  kfold=10
  kf = StratifiedKFold(n_splits=kfold, shuffle=True)
  feat_importance_df = pd.DataFrame()
  predictions_result = []

  for i, (train_index, test_index) in enumerate(kf.split(new_train, y)) :
    print('='*30, '{} of {} folds'.format(i+1, kfold), '='*30)
    x_train, x_val = new_train.iloc[train_index], new_train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

    clf.fit(x_train, y_train, eval_set=[(x_train, y_train),(x_val,y_val)],
            eval_metric=evaluate_macro_f1_lgb, categorical_feature=new_categorical_feats,)
    shap_values = shap.TreeExplainer(clf.booster_).shap_values(x_train)
    fold_importance_df = pd.DataFrame()
    fold_importance_df['feature']=x_train.columns
    fold_importance_df['shap_values'] = abs(np.array(shap_values)[:,:].mean(1).mean(0))
    fold_importance_df = pd.concat([feat_importance_df, fold_importance_df],axis=1)
    
    predicts_result.append(clf.predict(new_test))
    print_execution_time(start)

  return predicts_result, feat_importance_df

setting hyper parameters of lgb

In [ ]:
params = {'max_depth': 6,
         'learning_rate': 0.002,
          'colsample_bytree': 0.8,
          'subsample': 0.8,
          'min_split_gain': 0.02,
          'num_leaves': 48,
          'reg_alpha': 0.04,
          'reg_lambda': 0.073,
          'bagging_freq': 2,
          'min_child_weight': 40
         }

n_folds = 20
seed = 42
predicts_start, feat_importances_df = LGB_OOF(params, new_categorical_feats, n_folds, seed=42)

Feature importance

In [ ]:
fig, ax = pl.tsubplots(1,2,figsize=(20,20,))

feat_importance_df_shap = feat_importances_df.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

num_features = 50

sns.barplot(x=feat_importance_df_shap.shap_values[:num_features], y=feat_importance_df_shap.feature[:num_features], ax=ax[0])

ax[0].set_title('FEature importance based on shap values')

feat_importance_df = feat_importance_df.groupby('feature').mean().sort_values('feat_imp', ascending=False).reset_index()

num_features = 50
sns.barplot(feat_importances_df_shap.values[:num_features], feat_importances_df_shap.feature[:num_features], ax=ax[1])
ax[1].set_title('FEature importance based on feature importance form lgbm')
plt.show()

In [ ]:
submission['Target'] = np.array(predicts_results).mean(axis=0).round().atype(int)

submission.to_csv('submission_with_new_feature_set.csv',index=False)

#Randomized search

In [ ]:
optimized_params = None
lowest_cv = 1000
total_iteration = 100
for i in range(total_iteration) :
  print('-'*20, 'For {} of {} iterations'.format(i+1, total_iteration), '-'*20)
  learning_rate = np.random.rand() * 0.02
  n_folds =3

  num_class = len(np.unique(y))

  params = {}
  params['application'] = 'multiclass'
  params['metric'] = 'multi_logloss'
  params['num_class'] = num_class
  params['class_weight'] = 'balanced'
  params['num_leaves'] = np.random.randint(24, 48)
  params['max_depth'] = np.random.randint(5, 8)
  params['min_child_weight'] = np.random.randint(5, 50)
  params['min_split_gain'] = np.random.rand() * 0.09
  params['colsample_bytree'] = np.random.rand() * (0.9 - 0.1) + 0.1
  params['subsample'] = np.random.rand() * (1 - 0.8) + 0.8
  params['bagging_freq'] = np.random.randint(1, 5)
  params['bagging_seed'] = np.random.randint(1, 5)
  params['reg_alpha'] = np.random.rand() * 2
  params['reg_lambda'] = np.random.rand() * 2
  params['learning_rate'] = np.random.rand() * 0.02
  params['seed']  =1989

  d_Train = lgb.Dataset(data=new_train, label=y.values-1, categorical_feature = categorical_feats, free_raw_data = False)
  cv_results = lgb.cv(params=params, train_set = d_train, num_boost_round=10000, categorical_feature = new_categorical_Feats, nfold=n_folds,
                      stratified=True, shuffle=True, early_stopping_rounds=1, verbose_eval=1000)
  
  min_cv_results = min(cv_results['multi_logloss-mean'])

  if min_cv_results < lowest_cv :
    lowest_c = min_cv_results
    optimized_param = params

In [ ]:
submission['Target'] = np.array(predicts_result).mean(axis=0).round().astype(int)

submission.to_csv('submission_shap_randomized_search.csv', index=False)